In [42]:
from langgraph.graph import StateGraph, START, END
from langchain_community.chat_models import ChatOllama
from typing import TypedDict
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

In [43]:
load_dotenv()

True

In [44]:
# pick any model you have pulled via ollama (e.g., "llama2", "mistral")
model = ChatOllama(model="deepseek-r1:latest", temperature=0)

In [45]:
class LLMState(TypedDict):

    question: str
    answer: str

In [46]:
def llm_qa(state: LLMState) -> LLMState:

    # Extract the question from state
    question = state['question']

    # Form a prompt
    prompt = f'Answer the following {question}'

    # Ask that question to the LLM
    answer = model.invoke(prompt).content

    # Update the answer in the state
    state['answer'] = answer  # type:ignore

    return state

In [47]:
graph = StateGraph(LLMState)

# Add Nodes
graph.add_node('llm_qa', llm_qa)

# Add Edges
graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa', END)

# Compile graph
workflow = graph.compile()

In [ ]:
# Excute

initial_state = {'question': 'How far is moon from the earth ?'}

final_state = workflow.invoke(initial_state) # type:ignore

print(final_state['answer'])